### 1. Importaciones

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, lit, sum as spark_sum, countDistinct, month, quarter, year, weekofyear
from pyspark.sql.types import *
import os

### 2. Configuración de rutas y carga de datos

In [0]:
def create_spark_session():
    spark = SparkSession.builder.appName("ModeloEstrellaRestaurante").getOrCreate()
    return spark

def read_data(spark, input_data):
    df_categories = spark.read.csv(f"{input_data}/CATEGORIES.csv", header=True, inferSchema=True)
    df_customers = spark.read.csv(f"{input_data}/CUSTOMERS.csv", header=True, inferSchema=True)
    df_employees = spark.read.csv(f"{input_data}/EMPLOYEES.csv", header=True, inferSchema=True)
    df_orders = spark.read.csv(f"{input_data}/ORDERS.csv", header=True, inferSchema=True)
    df_prices = spark.read.csv(f"{input_data}/PRICES.csv", header=True, inferSchema=True)
    df_products = spark.read.csv(f"{input_data}/PRODUCTS.csv", header=True, inferSchema=True)
    df_sizes = spark.read.csv(f"{input_data}/SIZES.csv", header=True, inferSchema=True)
    return df_categories, df_customers, df_employees, df_orders, df_prices, df_products, df_sizes

In [0]:
# Configuración de rutas de entrada y salida
input_data = "dbfs:/FileStore/tables"
output_data = "/tmp/output/"

# Inicializar sesión de Spark
spark = create_spark_session()

# Leer datos
df_categories, df_customers, df_employees, df_orders, df_prices, df_products, df_sizes = read_data(spark, input_data)

### 3. Funciones para crear tablas de dimensiones

In [0]:
def create_dim_employees(df_employees):
    return df_employees.select(
        col("EMPLOYEE_ID").alias("id"),
        concat(col("FNAMEE"), lit(" "), col("LNAMEE")).alias("nombre_completo")
    ).distinct()

def create_dim_customers(df_customers):
    return df_customers.select(
        col("CUSTOMER_ID").alias("id"),
        concat(col("FNAMEC"), lit(" "), col("LNAMEC")).alias("nombre_completo")
    ).distinct()

def create_dim_products(df_products, df_categories, df_prices, df_sizes):
    return df_products \
    .join(df_categories.withColumnRenamed("DESCRIPTION", "categoria"), "CATEGORY_ID", "left") \
    .join(df_prices.withColumnRenamed("DESCRIPTION", "precio"), "PRICE_ID", "left") \
    .join(df_sizes.withColumnRenamed("DESCRIPTION", "tamaño"), "SIZE_ID", "left") \
    .select(
        df_products["PRODUCT_ID"].alias("id"),
        df_products["NAMEP"].alias("nombre_producto"),
        col("categoria"),
        col("precio"),
        col("tamaño")
    ).distinct()

def create_dim_time(df_orders):
    return df_orders.select(
        col("ORDER_DATE").alias("id_tiempo"),
        month(col("ORDER_DATE")).alias("mes"),
        quarter(col("ORDER_DATE")).alias("trimestre"),
        year(col("ORDER_DATE")).alias("semestre"),
        weekofyear(col("ORDER_DATE")).alias("dia_semana")
    ).distinct()

In [0]:
# Crear tablas de dimensiones
dim_employees = create_dim_employees(df_employees)
dim_customers = create_dim_customers(df_customers)
dim_products = create_dim_products(df_products, df_categories, df_prices, df_sizes)
dim_time = create_dim_time(df_orders)

### 4. Función para crear tabla de hechos

In [0]:
def create_hechos_ventas(df_orders, df_products, df_prices):
    return df_orders \
        .join(df_products, "PRODUCT_ID", "left") \
        .join(df_prices, "PRICE_ID", "left") \
        .withColumn("monto_total", col("AMOUNT") * col("DESCRIPTION").cast("float")) \
        .select(
            col("ORDER_ID").alias("hecho_id"),
            col("EMPLOYEE_ID"),
            col("CUSTOMER_ID"),
            col("PRODUCT_ID"),
            col("ORDER_DATE").alias("id_tiempo"),
            col("AMOUNT").alias("qty_venta"),
            "monto_total"
        )

In [0]:
# Crear tabla de hechos
hechos_ventas = create_hechos_ventas(df_orders, df_products, df_prices)

In [0]:
# Función para guardar DataFrames en formato Parquet
def save_parquet(df, output_path, partition_by=None):
    if partition_by:
        df.write.mode("overwrite").partitionBy(partition_by).parquet(output_path)
    else:
        df.write.mode("overwrite").parquet(output_path)

# Guardar tablas de dimensiones y hechos
save_parquet(dim_employees, os.path.join(output_data, "dim_employees"))
save_parquet(dim_customers, os.path.join(output_data, "dim_customers"))
save_parquet(dim_products, os.path.join(output_data, "dim_products"))
save_parquet(dim_time, os.path.join(output_data, "dim_time"))
save_parquet(hechos_ventas, os.path.join(output_data, "hechos_ventas"))

### 5. Preguntas de Negocio

In [0]:
# 1. Función para calcular el monto total de ventas por empleado

from pyspark.sql.functions import sum as spark_sum

def ventas_por_empleado(hechos_ventas, dim_employees):
    """
    Calcula el monto total de ventas por empleado.
    """
    return hechos_ventas \
        .join(dim_employees, hechos_ventas["EMPLOYEE_ID"] == dim_employees["id"]) \
        .groupBy("id", "nombre_completo") \
        .agg(spark_sum("monto_total").alias("monto_total_ventas")) \
        .orderBy("monto_total_ventas", ascending=False)

# 2. Función para analizar ventas por mes, trimestre y semestre
def ventas_por_tiempo(hechos_ventas, dim_time):
    """
    Analiza el monto total de ventas por mes, trimestre y semestre.
    """
    return hechos_ventas \
        .join(dim_time, hechos_ventas["id_tiempo"] == dim_time["id_tiempo"]) \
        .groupBy("mes", "trimestre", "semestre") \
        .agg(spark_sum("monto_total").alias("total_ventas")) \
        .orderBy("semestre", "trimestre", "mes")

# 3. Función para encontrar el día de la semana con mayor consumo
from pyspark.sql.functions import count

def dia_semana_mayor_consumo(hechos_ventas, dim_time):
    """
    Encuentra el día de la semana con el mayor número de ventas.
    """
    return hechos_ventas \
        .join(dim_time, hechos_ventas["id_tiempo"] == dim_time["id_tiempo"]) \
        .groupBy("dia_semana") \
        .agg(count("*").alias("conteo_ventas")) \
        .orderBy("conteo_ventas", ascending=False) \
        .limit(1)



# 4. Función para calcular la cantidad de clientes atendidos por mes
from pyspark.sql.functions import countDistinct

def cantidad_clientes_por_mes(hechos_ventas, dim_time):
    """
    Calcula la cantidad de clientes únicos atendidos por mes.
    """
    return hechos_ventas \
        .join(dim_time, hechos_ventas["id_tiempo"] == dim_time["id_tiempo"]) \
        .groupBy("mes") \
        .agg(countDistinct("CUSTOMER_ID").alias("cantidad_clientes")) \
        .orderBy("mes")

# 5. Función para encontrar los productos más y menos vendidos
from pyspark.sql.functions import lit

def productos_mas_y_menos_vendidos(hechos_ventas, dim_products):
    """
    Calcula los productos más y menos vendidos.
    """
    productos_venta = hechos_ventas \
        .join(dim_products, hechos_ventas["PRODUCT_ID"] == dim_products["id"]) \
        .groupBy("id", "nombre_producto", "categoria", "precio") \
        .agg(spark_sum("qty_venta").alias("total_vendido"))

    # Producto más vendido
    producto_mas_vendido = productos_venta.orderBy(col("total_vendido").desc()).limit(1) \
        .withColumn("demanda", lit("Mas Vendido"))
    
    # Producto menos vendido
    producto_menos_vendido = productos_venta.orderBy(col("total_vendido").asc()).limit(1) \
        .withColumn("demanda", lit("Menos Vendido"))
    
    # Unir los resultados
    return producto_mas_vendido.union(producto_menos_vendido)

In [0]:
# Ejecutar las funciones y mostrar los resultados
ventas_empleado_df = ventas_por_empleado(hechos_ventas, dim_employees)
ventas_empleado_df.show()

ventas_tiempo_df = ventas_por_tiempo(hechos_ventas, dim_time)
ventas_tiempo_df.show()

dia_mayor_consumo_df = dia_semana_mayor_consumo(hechos_ventas, dim_time)
dia_mayor_consumo_df.show()

clientes_mes_df = cantidad_clientes_por_mes(hechos_ventas, dim_time)
clientes_mes_df.show()

productos_ventas_df = productos_mas_y_menos_vendidos(hechos_ventas, dim_products)
productos_ventas_df.show()

+---+-------------------+------------------+
| id|    nombre_completo|monto_total_ventas|
+---+-------------------+------------------+
|  4|               null|            7900.0|
| 13|     Jung-kook Jeon|            5550.0|
|  3|      Pedro Infante|            3765.0|
|  9|               null|            3530.0|
|  5|               null|            3030.0|
| 11|               null|            2750.0|
| 10|               null|            1575.0|
|  2|Marco Antonio Solis|            1130.0|
| 12|  Vicente Fernandez|            1120.0|
|  1|               null|             740.0|
|  7|        Myke Towers|             550.0|
|  6|       Bryant Myers|             540.0|
|  8|               null|             500.0|
+---+-------------------+------------------+

+---+---------+--------+------------+
|mes|trimestre|semestre|total_ventas|
+---+---------+--------+------------+
|  7|        3|    2015|      1270.0|
|  6|        2|    2016|      1350.0|
|  5|        2|    2017|       170.0|
|  8| 

### 6. Métricas

In [0]:
# 1. Función para calcular el monto total de una orden
def monto_total_por_orden(hechos_ventas, dim_products):
    """
    Calcula el monto total de cada orden (precio * cantidad).
    """
    return hechos_ventas \
        .join(dim_products, hechos_ventas["PRODUCT_ID"] == dim_products["id"]) \
        .withColumn("monto_total", col("precio") * col("qty_venta")) \
        .select("hecho_id", "PRODUCT_ID", "qty_venta", "precio", "monto_total")

# 2. Función para contar la cantidad de clientes atendidos durante el mes
def cantidad_clientes_atendidos_mes(hechos_ventas, dim_time):
    """
    Cuenta los clientes únicos atendidos durante cada mes.
    """
    return hechos_ventas \
        .join(dim_time, hechos_ventas["id_tiempo"] == dim_time["id_tiempo"]) \
        .groupBy("mes") \
        .agg(countDistinct("CUSTOMER_ID").alias("cantidad_clientes")) \
        .orderBy("mes")


In [0]:
monto_total_orden_df = monto_total_por_orden(hechos_ventas, dim_products)
monto_total_orden_df.show()

clientes_atendidos_mes_df = cantidad_clientes_atendidos_mes(hechos_ventas, dim_time)
clientes_atendidos_mes_df.show()

+--------+----------+---------+------+-----------+
|hecho_id|PRODUCT_ID|qty_venta|precio|monto_total|
+--------+----------+---------+------+-----------+
|       1|         1|        3|   190|        570|
|       2|         2|        1|   170|        170|
|       3|         3|        1|   170|        170|
|       4|         4|        2|   160|        320|
|       5|        15|        5|   195|        975|
|       6|         6|        4|   165|        660|
|       7|         7|        3|   320|        960|
|       8|         8|        2|   240|        480|
|       9|        19|        2|   270|        540|
|      10|        10|        1|   240|        240|
|      11|        12|        1|   270|        270|
|      12|        14|        1|   270|        270|
|      13|        18|        2|   250|        500|
|      14|        22|        2|   275|        550|
|      15|        25|        6|   260|       1560|
|      16|        26|        6|   225|       1350|
|      17|        30|       10|

In [0]:
# Finalizar sesión de Spark
spark.stop()